https://github.com/BAMresearch/NFDI4IngScientificWorkflowRequirements

In [1]:
import os
from pyiron_base import Project

In [2]:
env_dict = {
    "preprocessing": "source/envs/preprocessing.yaml",
    "processing": "source/envs/processing.yaml",
    "postprocessing": "source/envs/postprocessing.yaml"
}

# Prepare Environments

In [3]:
pr = Project("demo")
pr.remove_jobs(recursive=True, silently=True)

In [4]:
for k, v in env_dict.items():
    pr.conda_environment.create(env_name=k, env_file=v)

# Preprocessing 

## generate mesh

In [5]:
def write_input(input_dict, working_directory):
    script_name = os.path.join(working_directory, "gmsh.sh")
    with open(script_name, "w") as f:
        f.writelines("gmsh -2 -setnumber domain_size " + str(input_dict["domain_size"]) + " unit_square.geo -o square.msh")
    os.chmod(script_name, 0o744)

In [6]:
gmsh = pr.create_job_step(
    job_name="gmsh",
    executable_str="./gmsh.sh",
    write_input_funct=write_input,
    input_dict={"domain_size": 2.0},
    conda_environment_path=pr.conda_environment.preprocessing,
    input_file_lst=["source/unit_square.geo"],
)

The job gmsh was saved and received the ID: 1


## convert to xdmf

In [7]:
meshio = pr.create_job_step(
    job_name="meshio",
    executable_str="meshio convert square.msh square.xdmf",
    conda_environment_path=pr.conda_environment.preprocessing,
    input_file_lst=[gmsh.files.square_msh],
)

The job meshio was saved and received the ID: 2


# Processing

## poisson

In [8]:
def collect_output(working_directory):
    with open(os.path.join(working_directory, "numdofs.txt"), "r") as f:
        return {"numdofs": int(f.read())}

In [9]:
poisson = pr.create_job_step(
    job_name="poisson",
    executable_str="python poisson.py --mesh square.xdmf --degree 2 --outputfile poisson.pvd --num-dofs numdofs.txt",
    collect_output_funct=collect_output,
    conda_environment_path=pr.conda_environment.processing,
    input_file_lst=["source/poisson.py", meshio.files.square_xdmf, meshio.files.square_h5],
)

The job poisson was saved and received the ID: 3


In [10]:
poisson.output["numdofs"]

357

# Postprocessing

## plot over line

In [11]:
pvbatch = pr.create_job_step(
    job_name="pvbatch",
    executable_str="pvbatch postprocessing.py poisson.pvd plotoverline.csv",
    conda_environment_path=pr.conda_environment.postprocessing,
    input_file_lst=["source/postprocessing.py", poisson.files.poisson_pvd, poisson.files.poisson000000_vtu],
)

The job pvbatch was saved and received the ID: 4


## substitute macros

In [12]:
def write_input(input_dict, working_directory):
    script_name = os.path.join(working_directory, "macros.sh")
    with open(script_name, "w") as f:
        f.writelines("python prepare_paper_macros.py --macro-template-file macros.tex.template --plot-data-path plotoverline.csv --domain-size " + str(input_dict["domain_size"]) + " --num-dofs " + str(input_dict["numdofs"]) + " --output-macro-file macros.tex")
    os.chmod(script_name, 0o744)

In [13]:
macros = pr.create_job_step(
    job_name="macros",
    executable_str="./macros.sh",
    write_input_funct=write_input,
    input_dict={"domain_size": 2.0, "numdofs": 100},
    conda_environment_path=pr.conda_environment.postprocessing,
    input_file_lst=["source/macros.tex.template", "source/prepare_paper_macros.py", pvbatch.files.plotoverline_csv],
)

The job macros was saved and received the ID: 5


## compile paper

In [14]:
tectonic = pr.create_job_step(
    job_name="tectonic",
    executable_str="tectonic paper.tex",
    conda_environment_path=pr.conda_environment.postprocessing,
    input_file_lst=["source/paper.tex", macros.files.macros_tex, pvbatch.files.plotoverline_csv],
)

The job tectonic was saved and received the ID: 6


# Overview


In [15]:
Project(".").job_table()

,id,status,chemicalformula,job,subjob,projectpath,project,timestart,timestop,totalcputime,computer,hamilton,hamversion,parentid,masterid
0,1,finished,None,gmsh,/gmsh,None,/home/jovyan/demo/,2024-02-09 06:06:39.997746,2024-02-09 06:06:45.173233,5.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dgicjnk9l#1,ExecutableContainerJob,0.4,None,None
1,2,finished,None,meshio,/meshio,None,/home/jovyan/demo/,2024-02-09 06:06:45.325307,2024-02-09 06:06:50.737994,5.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dgicjnk9l#1,ExecutableContainerJob,0.4,None,None
2,3,finished,None,poisson,/poisson,None,/home/jovyan/demo/,2024-02-09 06:06:50.891542,2024-02-09 06:07:12.690333,21.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dgicjnk9l#1,ExecutableContainerJob,0.4,None,None
3,4,finished,None,pvbatch,/pvbatch,None,/home/jovyan/demo/,2024-02-09 06:07:12.883348,2024-02-09 06:07:15.711106,2.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dgicjnk9l#1,ExecutableContainerJob,0.4,None,None
4,5,finished,None,macros,/macros,None,/home/jovyan/demo/,2024-02-09 06:07:15.861148,2024-02-09 06:07:16.582802,0.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dgicjnk9l#1,ExecutableContainerJob,0.4,None,None
5,6,finished,None,tectonic,/tectonic,None,/home/jovyan/demo/,2024-02-09 06:07:16.719532,2024-02-09 06:08:29.442532,72.0,pyiron@jupyter-pyiron-2ddev-2dnfdi4ing-5fpyiron-5fbase-2dgicjnk9l#1,ExecutableContainerJob,0.4,None,None
